### Spill
  
Å lage spill er en fin måte å øve seg på objektorientert programmering. For å lage spill kommer vi til å bruke `pygame`-pakken. 
    
Du kan finne dokumentasjonen til Pygame på [denne nettsiden](https://www.pygame.org/docs/).

#### Et grunnleggende eksempel.
  
Her er et enkelt eksempel som viser hvordan vi kan sette opp et spill med `pygame`.
 
```
import pygame
 
pygame.init()

screen = pygame.display.set_mode((500, 500)) # Setter skjermen til 500x500 piksler.

clock = pygame.time.Clock()

running = True

while running:
    # Avslutter løkken
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Fyller skjermen med hvit farge
    screen.fill("white")

    """
    Her skal vi putte spillets logikk, som å tegne figurer og oppdatere posisjonen deres.
    """

    # Oppdaterer hele skjermen
    pygame.display.flip()

    # Forsikrer at spillet kjører i maksimalt 60 FPS.
    clock.tick(60)

# Avslutter spillet
pygame.quit()
```

Dette spillet viser en hvit skjerm og avslutter programmet om vi lukker vinduet.